<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

<h2 style="color: #FFA07A;">4. Análise e explicabilidade dos agrupamentos (<i>clusters</i>) com inteligência artificial</h2>

In [1]:
import ipywidgets as widgets
import time
from IPython.display import display

# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# --- Texto formatado para o efeito de escrita ---
texto = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #FFA500; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
    <p>As decisões dos algoritmos podem ser difíceis de interpretar. Por isso, nesta secção aplicamos técnicas complementares de interpretabilidade (<i>Explainable Artificial Intelligence</i> – XAI), uma área da inteligência artificial que procura tornar os algoritmos mais transparentes, explicando de forma clara como tomam decisões.</p> 
    <p>Como o K-means não fornece, por si só, mecanismos para explicar a formação dos agrupamentos, recorremos às seguintes abordagens para explorar <u>como e porquê</u> os edifícios foram agrupados em dois <i>clusters</i>, utilizando técnicas de explicabilidade para compreender as decisões:</p> 
    <p> 🔸 <b><u>LIME (<i>Local Interpretable Model-Agnostic Explanations</i>)</u></b>: interpreta os resultados a nível local, evidenciando o impacto individual de cada variável nas previsões, considerando uma amostra específica nos dados de cada <i>cluster</i>. </p> 
    <p> 🔸 <b><u>Árvore de decisão</u></b>: oferece uma visão hierárquica e não linear sobre a influência das variáveis nos agrupamentos ou previsões.</p> 
    <p> 🔸 <b><u>Importância das variáveis (via Floresta Aleatória)</u></b>: calculada através de um modelo de Floresta Aleatória (Random Forest), esta análise identifica quais variáveis mais contribuíram para distinguir os grupos.</p> 
    <p> 🔸 Estas técnicas permitem uma compreensão mais aprofundada dos padrões identificados pelo K-means e serão integradas na análise global do modelo. </p> 
</div>
"""

# --- Efeito de escrita carácter a carácter (mantém o HTML) ---
typed = ""
for char in texto:
    typed += char
    output.value = typed
    time.sleep(0.005) # Ajustar velocidade

# --- Garantir que o texto completo é exibido no final ---
output.value = texto

HTML(value='<div></div>')

In [2]:
from IPython.display import Javascript, display
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Importar as bibliotecas ---
ipython = get_ipython()
ipython.run_line_magic("run", "1.preparacao_bibliotecas.ipynb")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Carregar os dados ---
input_pkl_path = "df_servicos_clusters.pkl"
with open(input_pkl_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)

df_servicos = data['df']  

# --- Mapeamento para nomes de variáveis mais legíveis ---
label_map = {
    'numero_servicos_proximos': 'Número total de serviços próximos',
    'pop_64_mais': 'População com 65 anos ou +',
    'distancia_media_servicos': 'Distância média aos serviços',
    'Centro Saude': 'Centro de saúde',
    'Farmacias': 'Farmácias',
    'Hospitais': 'Hospitais',
    'Supermercados': 'Supermercados',
    'Bancos': 'Bancos',
    'Parques e jardins': 'Parques ou jardins',
    'CTT': 'CTT'
}

# --- Preparar os dados para os modelos ---
X = df_servicos[['pop_64_mais', 'numero_servicos_proximos', 'distancia_media_servicos',
                 'Centro Saude', 'Farmacias', 'Hospitais',
                 'Supermercados', 'Bancos', 'Parques e jardins', 'CTT']]
y = df_servicos['cluster_kmeans']  # Usar os clusters do K-means

# --- Treinar o modelo Floresta Aleatória ---
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X, y)

# --- Inicializar o explicador LIME com nomes de variáveis legíveis ---
explainer = LimeTabularExplainer(
    X.values,
    feature_names=[label_map.get(col, col) for col in X.columns],
    class_names=[f"Cluster {i}" for i in sorted(y.unique())],
    mode="classification",
    discretize_continuous=True
)

# --- Treinar o modelo Árvore de decisão ---
dt_model = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_model.fit(X, y)

# --- Inicializar a aplicação Dash ---
app = Dash(__name__, suppress_callback_exceptions=True)

# --- Definir a apresentação da aplicação ---
app.layout = html.Div([
    html.H1("Modelo de avaliação com inteligência artificial explicável", style={
        'text-align': 'center',
        'color': 'white',
        'background-color': '#000',
        'border': '2px solid white',
        'padding': '10px',
        'font-weight': 'bold',
        'font-size': '32px'
    }),

    dcc.Tabs(id="tabs", value='tree-tab', children=[
        dcc.Tab(label='Árvore de decisão', value='tree-tab',
                style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
        dcc.Tab(label='LIME', value='lime-tab',
                style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
        dcc.Tab(label='Importância das variáveis', value='importance-tab',
                style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'})
    ]),

    html.Div(id='tabs-content')
])

# --- Função para elaborar a Árvore de Decisão em SVG ---
def gerar_arvore_svg():
    with NamedTemporaryFile(delete=False, suffix=".dot") as dot_file:
        export_graphviz(
            dt_model,
            out_file=dot_file.name,
            feature_names=[label_map.get(col, col) for col in X.columns],
            class_names=[f"Cluster {i}" for i in sorted(y.unique())],
            filled=True,
            rounded=True,
            special_characters=True,
            precision=0 
        )
        dot_file.close()

        svg_file = NamedTemporaryFile(delete=False, suffix=".svg")
        subprocess.run(["dot", "-Tsvg", dot_file.name, "-o", svg_file.name], check=True)

        with open(svg_file.name, "rb") as f:
            svg_content = f.read()

    return base64.b64encode(svg_content).decode('utf-8')

# --- Renderizar o conteúdo conforme o separador selecionado ---
@app.callback(Output('tabs-content', 'children'), Input('tabs', 'value'))
def render_tab_content(tab):
    if tab == 'tree-tab':
        svg_base64 = gerar_arvore_svg()
        return html.Div([
            html.Div([
                html.Img(src=f"data:image/svg+xml;base64,{svg_base64}")
            ], style={'text-align': 'center', 'overflow-x': 'scroll'})
        ])
    elif tab == 'lime-tab':
        return html.Div([
            dcc.Dropdown(
                id='lime-cluster-selector',
                options=[{'label': f'Cluster {i}', 'value': i} for i in sorted(y.unique())],
                placeholder="Selecione um cluster",
                style={'backgroundColor': 'white', 'color': 'black'}
            ),
            html.Div(id='lime-output', style={'padding': '10px', 'border': '1px solid #ccc',
                                              'borderRadius': '5px', 'backgroundColor': 'white'})
        ])
    elif tab == 'importance-tab':
        importances = rf_model.feature_importances_
        features = [label_map.get(col, col) for col in X.columns]
        fig = px.bar(x=importances, y=features, orientation='h',
                     labels={'x': 'Importância', 'y': 'Variáveis'})
        return html.Div([dcc.Graph(figure=fig)])

    return html.Div("Selecione uma aba para visualizar os resultados.")

# --- Apresentar a explicação LIME para o cluster selecionado ---
@app.callback(
    Output('lime-output', 'children'),
    Input('lime-cluster-selector', 'value')
)
def update_lime_output(cluster_selected):
    if cluster_selected is not None:
        try:
            cluster_indices = y[y == cluster_selected].index.tolist()
            if len(cluster_indices) == 0:
                return "Nenhuma instância encontrada para o cluster selecionado."

            np.random.seed(42)
            idx = np.random.choice(cluster_indices)

            explanation = explainer.explain_instance(
                X.iloc[idx].values,
                lambda x: rf_model.predict_proba(pd.DataFrame(x, columns=X.columns)),
                num_features=len(X.columns),
                labels=[cluster_selected]
            )
            return html.Iframe(
                srcDoc=explanation.as_html(),
                style={'width': '100%', 'height': '600px', 'border': 'none'}
            )
        except Exception as e:
            return f"Erro ao gerar a explicação com LIME: {e}"
    return "Selecione um cluster para visualizar a explicação com LIME."

# --- Encontrar uma porta de rede livre para o painel interativo ---
def encontrar_porta_livre():
    while True:
        porta = random.randint(8000, 9000)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", porta)) != 0:
                return porta

porta = encontrar_porta_livre()

# --- Lançar a aplicação interativa quando o ficheiro for executado ---
if __name__ == '__main__':
    app.run(debug=False, port=porta)

# ---- Mensagem final---
print("\033[92m[INFO] Análise concluída. Você pode prosseguir.\033[0m")

<IPython.core.display.Javascript object>

[INFO] Análise concluída. Você pode prosseguir.


In [1]:
#
import ipywidgets as widgets
import time
from IPython.display import display

# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output2 = widgets.HTML(value="<div></div>")
display(output2)

# --- Texto formatado para o efeito de escrita ---
texto_restante = """
<p style="text-align: justify;">
<p><b>Explicação:</b></p>
🔹 <strong><u>Árvore de Decisão:</u></strong> revela que o <strong>número de serviços próximos</strong> é a variável com maior influência na segmentação, surgindo de forma predominante nos primeiros níveis da árvore. Esta variável é essencial para distinguir os dois <i>clusters</i>. O <i><strong>Cluster 0</strong></i> representa edifícios localizados em áreas com <strong>menor oferta de serviços nas proximidades</strong>, enquanto o <i><strong>Cluster 1</strong></i> agrupa áreas com <strong>maior concentração e diversidade de serviços urbanos</strong>.</p>
<p>A variável <strong>população</strong> surge apenas em níveis mais profundos, o que indica que o <strong>fator demográfico</strong> tem um papel secundário em comparação com a acessibilidade aos serviços. Outras variáveis, como <em>hospitais</em>, <em>bancos</em> e <em>centros de saúde</em>, também contribuem para a classificação, mas atuam de forma complementar, refinando a segmentação em casos específicos.</p>
<p><strong><u>LIME (Local Interpretable Model-Agnostic Explanations)</u></strong>: permite perceber como o modelo tomou uma decisão em relação a um edifício específico. Para isso, apresenta três gráficos principais, cada um com uma função distinta, facilitando a compreensão, mesmo para quem nunca utilizou esta técnica:</p>
<ul>
  <li><strong>Gráfico de probabilidade de predição:</strong> mostra a probabilidade de o edifício pertencer a cada um dos <i>clusters</i>. Por exemplo, se o modelo indicar 100% de probabilidade para o <i><strong>Cluster 0</strong></i> e 0% para o <i><strong>Cluster 1</strong></i>, <strong>isto significa</strong> que o modelo tem total confiança de que o edifício pertence ao <i><strong>Cluster 0</strong></i>.</li>

  <li><strong>Gráfico das variáveis mais relevantes (localizado no centro da figura):</strong> mostra as variáveis que mais influenciaram a decisão do modelo para aquele edifício. As barras <span style="color:orange;"><strong>laranja</strong></span> indicam uma influência a favor do <i><strong>Cluster 1</strong></i>, enquanto as barras <span style="color:blue;"><strong>azuis</strong></span> indicam uma influência a favor do <i><strong>Cluster 0</strong></i>. Quanto maior for a barra, <strong>maior é</strong> o impacto dessa variável na decisão final. <strong>Nota:</strong> Mesmo que a maioria das barras seja laranja, o edifício pode ser classificado no <i><strong>Cluster 0</strong></i>. <strong>Isto</strong> acontece porque o LIME destaca as variáveis que mais poderiam mudar a decisão, e não todas as que o modelo considerou.</li>

  <li><strong>Coluna «valor» (valores das variáveis):</strong> mostra o valor real de cada variável para o edifício analisado. Por exemplo, se a variável <em>"bancos"</em> tiver o valor <strong>9</strong>, significa que existem 9 bancos nas proximidades (1.5 km). O LIME indica como esse valor influenciou a classificação no respetivo <i>cluster</i>.</li>
</ul>
<p style="text-align: justify;">
🔹 <b><u>Importância das variáveis:</u></b> destaca que as variáveis mais relevantes são: o <i>número de serviços próximos</i>, <strong>seguido</strong> por variáveis como supermercados, bancos, CTT e farmácias. Por outro lado, variáveis como a distância média aos serviços, hospitais e parques ou jardins têm uma influência menor. Assim, conclui-se que a presença e diversidade de serviços de proximidade são os principais critérios na definição dos <i>clusters</i>.
</p>
<p style="text-align: justify;">
🔹 <b>Conclusão:</b> Tanto a <b>Árvore de Decisão</b> como a <b>Floresta Aleatória</b> (<b><u>importância das variáveis</u></b>) atribuem maior peso <strong>à acessibilidade e à diversidade de serviços</strong> na definição dos <i>clusters</i>. A variável <b>população com 65 anos ou mais</b> não se destacou como um critério relevante, aparecendo apenas em níveis mais baixos da Árvore de Decisão. Isto sugere que a <b>presença e concentração de serviços</b> <strong>foram os fatores</strong> mais determinantes na segmentação realizada pelo algoritmo <b>K-means</b>.
</p>
</div>
"""
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #6A0DAD; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""

# --- Efeito de escrita carácter a carácter (mantém o HTML) ---
for palavra in texto_restante.split():
    texto_html += palavra + " "
    output2.value = texto_html + "</div>"
    time.sleep(0.10)   # Ajustar velocidade

# --- Garantir que o texto completo é exibido no final ---
output2.value = texto_html + "</div>"

HTML(value='<div></div>')

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

In [4]:
import os
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets

# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Pasta específica para guardar as respostas ---
pasta_respostas = ".../respostas"
os.makedirs(pasta_respostas, exist_ok=True)  

# --- Questão de escolha múltipla ---
pergunta = widgets.RadioButtons(
    options=[
        "a) Bancos.",
        "b) Número total de serviços próximos.",
        "c) Farmácias."
    ],
    description='Resposta:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
    value=None
    )

# --- Botão para gravar a resposta ---
botao_gravar = widgets.Button(description='Gravar resposta', button_style='info')
output = widgets.Output()

# --- Função para gravar a resposta selecionada ---
def gravar_resposta(b):
    resposta = pergunta.value

    if not resposta:
        with output:
            clear_output()
            print("Por favor, selecione uma resposta antes de gravar.")
        return

    dados = {
        "Resposta": [resposta],
        "Data": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    }

    ficheiro = os.path.join(pasta_respostas, "respostas_LIME.xlsx")
    df = pd.DataFrame(dados)

    if not os.path.isfile(ficheiro):
        with output:
            clear_output()
            print(f"O ficheiro {ficheiro} não existe. A criar o ficheiro...")
        df.to_excel(ficheiro, index=False, engine='openpyxl')
        with output:
            clear_output()
            print(f"Ficheiro criado com sucesso em: {ficheiro}")
    else:
        df_existente = pd.read_excel(ficheiro, engine='openpyxl')
        df_final = pd.concat([df_existente, df], ignore_index=True)
        df_final.to_excel(ficheiro, index=False, engine='openpyxl')
        with output:
            clear_output()
            print(f"Resposta adicionada ao ficheiro {ficheiro}")

    # Limpar a seleção e mostrar mensagem de confirmação
    with output:
        clear_output()
        print("Resposta gravada com sucesso! Obrigado.")
        pergunta.value = None

botao_gravar.on_click(gravar_resposta)

# --- Instrução antes de apresentar a questão ---
display(widgets.HTML("""
<b style="font-size: 18px;">Por favor, responda à questão:</b><br><br>
<b style="font-size: 16px;">3. Com base na explicação dada pelo LIME, qual das seguintes variáveis teve maior impacto na classificação desta amostra no agrupamento (Cluster 1)?</b><br>
"""))

display(pergunta, botao_gravar, output)

<IPython.core.display.Javascript object>

HTML(value='\n<b style="font-size: 18px;">Por favor, responda à questão:</b><br><br>\n<b style="font-size: 16p…

RadioButtons(description='Resposta:', layout=Layout(width='600px'), options=('a) Bancos.', 'b) Número de servi…

Button(button_style='info', description='Gravar resposta', style=ButtonStyle())

Output()

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

Seguinte: [Acesso serviços no contexto cidade 15 minutos](5.ipynb)

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>